# Twitter Sentiment Analysis

In [37]:
import twitter
import pandas as pd
import numpy as np

In [38]:
### read csv input file:
df_source = pd.read_csv("/Users/adrientalbot/Desktop/Studies/Imperial/Capstone/capital_pilot/RefData.csv")
buzzwords = df_source['Buzzwords'].iloc[0]
buzzwords_list = buzzwords.split(",")
buzzwords_list

['Artificial Intelligence', 'Big data', 'FinTech']

### Source

https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed

### Authenticating Twitter API

In [39]:
# Authenticating our twitter API credentials
twitter_api = twitter.Api(consumer_key='f2ujCRaUnQJy4PoiZvhRQL4n4',
                        consumer_secret='EjBSQirf7i83T7CX90D5Qxgs9pTdpIGIsVAhHVs5uvd0iAcw5V',
                        access_token_key='1272989631404015616-5XMQkx65rKfQU87UWAh40cMf4aCzSq',
                        access_token_secret='emfWcF8fyfqoyywfPCJnz4jXt6DFXfndro59UK9IMAMgy')

# Test authentication to make sure it was successful
print(twitter_api.VerifyCredentials())

{"created_at": "Tue Jun 16 20:29:26 +0000 2020", "default_profile": true, "default_profile_image": true, "id": 1272989631404015616, "id_str": "1272989631404015616", "name": "Nicola Osrin", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "NicolaOsrin"}


### Building the Test Set

In [40]:
#We first build the test set, consisting of only 100 tweets for simplicity. 
#Note that we can only download 180 tweets every 15min.
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [41]:
#Testing out fetching the test set. The function below prints out the first 5 tweets in our test set.
search_term = input("Enter a search keyword:")
testDataSet = buildTestSet(search_term)

print(testDataSet[0:4])

Enter a search keyword:FinTech
Fetched 100 tweets for the term FinTech
[{'text': 'The #BISInnovationHub is pleased to announce a strategic partnership with @federalreserve @NewYorkFed to further th… https://t.co/BWlrafHHMf', 'label': None}, {'text': '🚨URGENTE🚨\n✅Guerra🥊de Lobos🐺 \n✅Un grupo de grandes fondos aprobaron la nueva oferta de Argentina \n✅Gramercy, Fintech… https://t.co/25vMDYUFZU', 'label': None}, {'text': 'The #BISInnovationHub is pleased to work with @ecb, @banquedefrance and @bundesbank on a new ECB/Eurosystem Hub cen… https://t.co/MB9XdNlsmi', 'label': None}, {'text': 'RT @financeinnov: Présentation par @mPtachek des métiers #Fintech les plus attractifs. https://t.co/CbD2RmnK9Z', 'label': None}]


In [42]:
testDataSet[0]

{'text': 'The #BISInnovationHub is pleased to announce a strategic partnership with @federalreserve @NewYorkFed to further th… https://t.co/BWlrafHHMf',
 'label': None}

In [43]:
#df = pd.DataFrame(list())
#df.to_csv('tweetDataFile.csv')

### Building the Training Set

We will be using a downloadable training set, consisting of 5,000 tweets. These tweets have already been labelled as positive/negative. We use this training set to calculate the posterior probabilities of each word appearing and its respective sentiment.

In [44]:
#As Twitter doesn't allow the storage of the tweets on personal drives, we have to create a function to download
#the relevant tweets that will be matched to the Tweet IDs and their labels, which we have.

def buildTrainingSet(corpusFile, tweetDataFile, size):
    import csv
    import time
    
    count = 0
    corpus = []
    
    with open(corpusFile,'r') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
            if count <= size: 
                corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})
                count += 1
            else: 
                break

    rate_limit = 180
    sleep_time = 900/180
    
    trainingDataSet = []
    
    for tweet in corpus:
        try:
            status = twitter_api.GetStatus(tweet["tweet_id"])
            print("Tweet fetched" + status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(sleep_time) 
        except: 
            continue
    # now we write them to the empty CSV file
    with open(tweetDataFile,'w') as csvfile:
        linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingDataSet

In [45]:
#This function is used to download the actual tweets. It takes hours to run and we only need to run it once
#in order to get all 5,000 training tweets. The 'size' parameter below is the number of tweets that we want to
#download. If 5,000 => set size=5,000

'''
corpusFile = "corpus.csv"
tweetDataFile = "tweetDataFile.csv"

trainingData = buildTrainingSet(corpusFile, tweetDataFile, 5000)
'''

#When this code stops running, we will have a tweetDataFile.csv full of the tweets that we downloaded.

'\ncorpusFile = "corpus.csv"\ntweetDataFile = "tweetDataFile.csv"\n\ntrainingData = buildTrainingSet(corpusFile, tweetDataFile, 5000)\n'

In [46]:
#This line counts the number of tweets and their labels in the Corpus.csv file that we originally downloaded
corp = pd.read_csv("corpus.csv", header = 0 , names = ['topic','label', 'tweet_id'] )
corp['label'].value_counts()

neutral       2503
irrelevant    1786
negative       654
positive       569
Name: label, dtype: int64

In [47]:
#As a check, we look at the first 5 lines in our new tweetDataFile.csv
trainingData_copied = pd.read_csv("tweetDataFile.csv", header = None, names = ['tweet_id', 'text', 'label', 'topic'])
trainingData_copied.head()

,tweet_id,text,label,topic
0,126415614616154112,Now all @Apple has to do is get swype on the i...,positive,apple
1,126402758403305474,Hilarious @youtube video - guy does a duet wit...,positive,apple
2,126397179614068736,@RIM you made it too easy for me to switch to ...,positive,apple
3,126379685453119488,The 16 strangest things Siri has said so far. ...,positive,apple
4,126377656416612353,Great up close & personal event @Apple tonight...,positive,apple


In [48]:
len(trainingData_copied)

3651

In [49]:
#We check the number of tweets by each label in our training set
trainingData_copied['label'].value_counts()

neutral       1828
irrelevant     993
negative       435
positive       395
Name: label, dtype: int64

In [50]:
df = trainingData_copied.copy()
lst_labels = df['label'].unique()
count_rows_keep = df['label'].value_counts().min()

neutral_df = df[df['label'] == 'neutral'].sample(n= count_rows_keep , random_state = 2)
irrelevant_df = df[df['label'] == 'irrelevant'].sample(n= count_rows_keep , random_state = 3)
negative_df = df[df['label'] == 'negative'].sample(n= count_rows_keep , random_state = 1)
positive_df = df[df['label'] == 'positive'].sample(n= count_rows_keep , random_state = 1)

lst_df = [neutral_df, irrelevant_df, negative_df, positive_df]

trainingData_copied = pd.concat(lst_df)
trainingData_copied['label'].value_counts()

positive      395
irrelevant    395
negative      395
neutral       395
Name: label, dtype: int64

In [51]:
'''
def oversample(df):
    lst_labels = df['label'].unique()
    for x in lst_labels:
        if len(df[df['label'] == x]) < df['label'].value_counts().max():
            df=df.append(df[df['label'] == x]*((df['label'].value_counts().max())/ len(df[df['label'] == 'x']))) 
    return df
'''

"\ndef oversample(df):\n    lst_labels = df['label'].unique()\n    for x in lst_labels:\n        if len(df[df['label'] == x]) < df['label'].value_counts().max():\n            df=df.append(df[df['label'] == x]*((df['label'].value_counts().max())/ len(df[df['label'] == 'x']))) \n    return df\n"

In [52]:
'''
def undersample(df):
    lst_labels = df['label'].unique()
    for x in lst_labels:
        if len(df[df['label'] == 'x']) > df['label'].value_counts().min():
            count_rows_keep = df['label'].value_counts().min()
            sample = df[df['label'] == 'x'].sample(n= count_rows_keep , random_state = 1)
            index_drop = pd.concat([df[df['label'] == 'x'], sample]).drop_duplicates(keep=False).index
            df = df.drop(index_drop)
    return df
'''

"\ndef undersample(df):\n    lst_labels = df['label'].unique()\n    for x in lst_labels:\n        if len(df[df['label'] == 'x']) > df['label'].value_counts().min():\n            count_rows_keep = df['label'].value_counts().min()\n            sample = df[df['label'] == 'x'].sample(n= count_rows_keep , random_state = 1)\n            index_drop = pd.concat([df[df['label'] == 'x'], sample]).drop_duplicates(keep=False).index\n            df = df.drop(index_drop)\n    return df\n"

In [53]:
trainingData_copied = trainingData_copied.to_dict('records')

### Pre-processing

Here we use the NLTK library to filter for keywords and remove irrelevant words in tweets. We also remove punctuation and things like images (emojis) as they cannot be classified using this model.

In [54]:
import re #a library that makes parsing strings and modifying them more efficient
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import nltk #Natural Processing Toolkit that takes care of any processing that we need to perform on text 
            #to change its form or extract certain components from it.
    
#nltk.download('popular') #We need this if certain nltk libraries are not installed. 

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

In [55]:
#Here we call the function to pre-process both our training and our test set. 
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData_copied)
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

### Building the Naive Bayes Classifier

We apply a classifier based on Bayes' Theorem, hence the name. It allows us to find the posterior probability of an event occuring (in this case that event being the sentiment- positive/neutral or negative) is reliant on another probabilistic background that we know. 

The posterior probability is calculated as follows:
$P(A|B) = \frac{P(B|A)\times P(A)}{P(B)}$

The final sentiment is assigned based on the highest probability of the tweet falling in each one.

#### To read more about Bayes Classifier in the context of classification:
https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html

### Build the vocabulary

In [56]:
#Here we attempt to build a vocabulary (a list of words) of all words present in the training set.

import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

#This function generates a list of all words (all_words) and then turns it into a frequency distribution (wordlist)
#The word_features is a list of distinct words, with the key being the frequency of each one.

### Matching tweets against our vocabulary

Here we go through all the words in the training set (i.e. our word_features list), comparing every word against the tweet at hand, associating a number with the word following:

label 1 (true): if word in vocabulary occurs in tweet

label 0 (false): if word in vocabulary does not occur in tweet

In [57]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features 

### Building our feature vector

In [58]:
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

The feature vector is a list of dictionaries where each dictionary compares the list of words in a specific tweet against the Vocabulary list of unique words as well as the label of each tweet. 

We will input the feature vector in the Naive Bayes Classifier, which will calculate the posterior probability given the prior probability that a randomly chosen observation is associated with a certain label, and the likelihood which is the probability of having a specific word in a tweet given a specific label (density function of X for an observation that comes from the k class/label) and you have to calculate this probability for each word and for each label to get posterior probability)

### Train the Naives Bayes Classifier

In [59]:
#This line trains our Bayes Classifier
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)


In [63]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]


In [65]:
NBResultLabels.count('positive')

18

In [67]:
100*NBResultLabels.count('positive')/len(NBResultLabels)

18.0

In [74]:
twitter_results = {'positive count': NBResultLabels.count('positive'), 'negative count': NBResultLabels.count('negative'), 'neutral count': NBResultLabels.count('neutral'), 'irrelevant count': NBResultLabels.count('irrelevant')}
twitter_results

{'positive count': 18,
 'negative count': 11,
 'neutral count': 41,
 'irrelevant count': 30}

## Test Classifier 

In [61]:
#We now run the classifier and test it on 100 tweets previously downloaded in the test set, on our specified keyword.

# NBResultLabels is the list of the labels for the tweets in the test set: 
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
    print("Number of negative comments = " + str(NBResultLabels.count('negative')))
    print("Number of positive comments = " + str(NBResultLabels.count('positive')))
    print("Number of neutral comments = " + str(NBResultLabels.count('neutral')))
    print("Number of irrelevant comments = " + str(NBResultLabels.count('irrelevant')))

Overall Positive Sentiment
Positive Sentiment Percentage = 11.0%


In [62]:
len(preprocessedTestSet)

100

In [63]:
import plotly.graph_objects as go

sentiment = ["Negative","Positive","Neutral", "Irrelevant"]

fig = go.Figure([go.Bar(x=sentiment, y=[str(NBResultLabels.count('negative')), str(NBResultLabels.count('positive')), str(NBResultLabels.count('neutral')), str(NBResultLabels.count('irrelevant'))])])
fig.update_layout(title_text='Sentiment Results for Specific Keyword')

fig.update_layout(template = 'simple_white',
    title_text='Twitter Sentiment Results', 
    yaxis=dict(
    title='Percentage (%)',
    titlefont_size=16,
    tickfont_size=14,) ,
        
        
)

fig.show()


### TBC:
- Retrieve tweets about keyword, not from keyword (username)

In [62]:
'''

twitters_and_news = []
for buzzword in buzzwords:
    twitters_and_news.append(sp.sentiment_analysis(buzzword,
                                                   tw['consumer_key'],
                                                   tw['consumer_secret'],
                                                   tw['access_token_key'],
                                                   tw['access_token_secret'],
                                                   news['secret_api']))
                                                   
'''

"\n\ntwitters_and_news = []\nfor buzzword in buzzwords:\n    twitters_and_news.append(sp.sentiment_analysis(buzzword,\n                                                   tw['consumer_key'],\n                                                   tw['consumer_secret'],\n                                                   tw['access_token_key'],\n                                                   tw['access_token_secret'],\n                                                   news['secret_api']))\n                                                   \n"